In [2]:
!pip install nltk

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 1.5MB 432kB/s ta 0:00:011
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/chinmay/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [1]:
#installing all the libraries needed for the task
import pandas as pd
import json
import nltk
from pandas.io.json import json_normalize
nltk.download('stopwords')
nltk.download('punkt')
import re  
from nltk.corpus import stopwords
stops1 = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chinmay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/chinmay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
! pip install gensim
! pip install seaborn
! pip install elasticsearch
! pip install esengine
! pip install ipdb

In [4]:
# %cd /home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts/github-code-guided-research/
from Vocabulary import Vocab

In [5]:
%cd /home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts/

/home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts


In [6]:
from preprocess import *
from sub_find import *
from true_subs import *
from constants import *
# from graph_evaluation import *
# from Vocabulary import Vocab
import pickle

In [7]:
from recipe import Recipe
def load_data_from_file(filename=None, ing_file_name=None, local=True, save=False):
    return load_data(file_name=filename, ing_file_name=ing_file_name, local=local, save=save)
    

In [8]:
import os
os.chdir('raw_data')
recipes = load_data_from_file("100k/100ksample", ing_file_name = "dbpedia_ingredients.txt",local=True, save=False )
clean_recipes = [x.proccessed_ing_list for x in recipes]
recipes = load_data_from_file("100k/100ksample2", "dbpedia_ingredients.txt")
sample100k2 = [x.proccessed_ing_list for x in recipes]
clean_recipes.extend(sample100k2) 
type(clean_recipes)

list

In [9]:
# currently looking only on the second set of elements
total_number_of_recipes = len(recipes)
print(total_number_of_recipes)

100000


In [10]:
titles_present = set()
for reduced_recipe_object in recipes:
  titles_present.add(str(reduced_recipe_object.id))

In [11]:
vars(recipes[0])

{'id': 'Cajun Spice Blend',
 'title': 'Cajun Spice Blend',
 'instructions_text': 'mix together and store in airtight container and use as wanted.',
 'ingredients_text': '1 tablespoon garlic powder 1 tablespoon onion powder 1 tablespoon sugar 1 teaspoon salt 1 teaspoon pepper 1 teaspoon dried basil 1 teaspoon paprika 12 teaspoon cayenne pepper',
 'proccessed_ing_list': ['garlic powder',
  'onion powder',
  'sugar',
  'cayenne pepper']}

In [12]:
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
final_dataset[0]

(['garlic powder', 'onion powder', 'sugar', 'cayenne pepper'],
 'Cajun Spice Blend')

In [11]:
# let us use nltk to perform stemming. That way, it is possible to get the matches better
# eg muffins and muffin

In [13]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
# type(titles_present)

In [14]:
def stem_title(word):
    return stemmer.stem(word)


In [15]:
# an example
stem_title('Carsons Chicken Wellington W/ Exotic Mushroom Sherry Sauce')

'carsons chicken wellington w/ exotic mushroom sherry sauc'

In [ ]:
updated_title = [stem_title(title) for title in titles_present]

In [ ]:
with open('recipe_titles_stemmed.txt', 'w') as file:
    for item in updated_title:
        file.write("%s\n" % item)

In [21]:
my_list = 'cake,pasta,custard,rolls,stew,casserole,sauce,soup,burgers,pizza,salad,muffins,steaks,fish,ham'.split(",")

In [ ]:
vindhya_list = ['Cake', 'Salad', 'Noodles', 'Hummus', 'Burger', 'Steak', 'Pie', 'Chicken', 'Custard', 'Chips',
                'Casserole', 'Pasta', 'Pizza', 'Omelette', 'soup', 'sauce']

In [ ]:
# check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    counter = counter +  any (stem_title(title) in stem_title(reduced_recipe_object.id) for title in vindhya_list)
    
counter

31123

In [ ]:
percentage_covered = counter / total_number_of_recipes * 100
print(percentage_covered)

31.123


# Eventually we will be using the cleaned recipe list for our evaluation

In [ ]:
# so sample 100k2
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
only_ingredients = [i[0] for i in final_dataset]

In [ ]:
type(final_dataset[0])

tuple

## TODO: Fix the code here. I just used glove pretrined so things can be ignored for a while here

In [ ]:
ingredient_vocabulary = Vocab()
for recipe in only_ingredients:
    for ingredients in recipe:
        ingredient_vocabulary.add_word(ingredients)

In [ ]:
# Let us see the vocabulary size once
ingredient_vocabulary.max_idx 

1152

In [25]:
ingredient_vocabulary.get_word(100)

'hamburger'

In [ ]:
import pandas as pd
df = pd.DataFrame(final_dataset, columns =['Ingredients', 'Recipe_id'])

In [ ]:
df.iloc[5]

Ingredients    [orange, clove, apple, water, cinnamon]
Recipe_id                     The Spiced Cider Project
Name: 5, dtype: object

## We plan to use the gensim model as well. We create another pickle file with the Word2Vec embedding

In [28]:
! wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
! gzip -d GoogleNews-vectors-negative300.bin.gz


--2019-12-03 23:57:42--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.242.214
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.242.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1,5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1,53G  1,65MB/s    in 32m 57s 

2019-12-04 00:30:41 (814 KB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



MemoryError: Unable to allocate array with shape (3000000, 300) and data type float32

In [ ]:
import gensim
pretrained_embedding = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# Just a sample check to indicate things are working as expected
pretrained_embedding.vocab['orange'].index

In [ ]:
# This is more of a hack
def recipe_vocab_on_list(ingedient_list):
    result_list = []
    for ingedient in ingedient_list:
        # If multi word ingredient, chose the first word
        ingedient = ingedient.split(" ")[0]
        try:
            result_list.append(pretrained_embedding.vocab[ingedient].index)
        except KeyError:
            result_list.append(pretrained_embedding.vocab['unk'].index)
    return result_list
    

In [ ]:
df['Ingredient_Numeric'] = df.Ingredients.apply(recipe_vocab_on_list)

In [ ]:
df.iloc[5]

In [ ]:
recipe_vocabulary = Vocab()
for recipe in vindhya_list:
    recipe_vocabulary.add_word(stemmer.stem(recipe))

In [ ]:
recipe_vocabulary

In [ ]:
len(recipe_vocabulary.word_2_idx)

In [ ]:
recipe_vocabulary.check_any_word_in_vocab(stemmer.stem('The Spiced Cake Project'))
# recipe_vocabulary.get_idx('cake')

In [ ]:
def get_recipe_idx(recipe):
    recipe_stemmed = stemmer.stem(recipe)
    lookup_result = recipe_vocabulary.check_any_word_in_vocab(recipe_stemmed)
    if lookup_result[0]:
        return recipe_vocabulary.get_idx(lookup_result[1])
    return 0

In [43]:
get_recipe_idx('The Spiced Cake Project')

1

In [ ]:
df['Recipe_id_numeric'] = df.Recipe_id.apply(get_recipe_idx)

In [ ]:
df.iloc[5]

In [39]:
df.Recipe_id_numeric.value_counts()

0     78650
11     5466
8      3373
7      3011
2      1617
6      1442
10     1091
12     1016
1       973
13      897
5       761
15      733
14      546
3       212
4       120
9        92
Name: Recipe_id_numeric, dtype: int64

In [40]:
len(df)

100000

## I am pickling both the vocabularies which would be useful later

In [108]:
with open('vocab_combined.pkl','wb') as file:
    pickle.dump(recipe_vocabulary, file)
    pickle.dump(ingredient_vocabulary, file)

## Let us drop the empty entries

In [ ]:
def check_empty(x):
    return len(x) == 0
    

In [ ]:
processed_df = df.drop(df[df.Ingredient_Numeric.apply(check_empty)].index)

In [ ]:
assert sum(processed_df.Ingredient_Numeric.apply(check_empty)) == 0

In [ ]:
# I am just saving the file here for reference later
pd.to_pickle(processed_df, './processed_df_pretrained.pkl')

## Let us build the two set of data using sklearn dataloaders 

In [44]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(processed_df, test_size = 0.2, random_state = 0)

# This is the Neural Network portion

In [45]:
# I am assuming that we are able to get the categories here
# Now this should become a LSTM based model which will try and do binary prediction
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.utils.data.dataloader as dataloader
from torch.nn.utils.rnn import pad_sequence

In [46]:
# Some constants
batch_size = 5

In [47]:
# https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of tuple (src_seq, trg_seq).
            - src_seq: torch tensor of shape (?); variable length.
            - trg_seq: torch tensor of shape (?); variable length.
    Returns:
        src_seqs: torch tensor of shape (batch_size, padded_length).
        src_lengths: list of length (batch_size); valid length for each padded source sequence.
        trg_seqs: torch tensor of shape (batch_size, padded_length).
        trg_lengths: list of length (batch_size); valid length for each padded target sequence.
    """
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
#     print(data[0]) # list of tuples
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs, trg_seqs = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)
    # target sequence for us is a single tensor so we do not need to 
    # merge it
    #trg_seqs, trg_lengths = merge(trg_seqs)
    trg_seqs = torch.as_tensor(trg_seqs)
    return src_seqs, src_lengths, trg_seqs #, trg_lengths


In [48]:
class RecipeData(data.Dataset):
    
    def __init__(self, df):
        super(RecipeData, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = torch.as_tensor(self.df.Ingredient_Numeric.iloc[idx])
        y = torch.as_tensor(self.df.Recipe_id_numeric.iloc[idx])
        return X,y
    

In [49]:
train_dataset = RecipeData(df_train)
test_dataset = RecipeData(df_test)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                               drop_last=True) # Done for cases when num_samples not exact multiple
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, # of the batch_size
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                              drop_last=True)

In [50]:
df.Recipe_id_numeric[4]

5

In [51]:
train_dataset[0]

(tensor([ 65,  33, 140,  36]), tensor(0))

In [52]:
for (X,X_len,y) in train_data_loader:
    print(X)
    print(X_len)
    print(y)
    break

tensor([[ 19,   3, 174,  39,  95,  30,  52,  18, 225,   8,  38],
        [130, 281,  38, 721,  24,  69, 129, 287, 278, 131,   0],
        [486, 331,  23, 187,  80, 401, 185, 186, 341,   0,   0],
        [131, 167, 151, 263,  61, 292,  52,  30,   0,   0,   0],
        [129,   3,  94, 137, 196,   0,   0,   0,   0,   0,   0]])
[11, 10, 9, 8, 5]
tensor([1, 0, 0, 0, 0])


In [53]:
X

tensor([[ 19,   3, 174,  39,  95,  30,  52,  18, 225,   8,  38],
        [130, 281,  38, 721,  24,  69, 129, 287, 278, 131,   0],
        [486, 331,  23, 187,  80, 401, 185, 186, 341,   0,   0],
        [131, 167, 151, 263,  61, 292,  52,  30,   0,   0,   0],
        [129,   3,  94, 137, 196,   0,   0,   0,   0,   0,   0]])

In [54]:
class RecipePredictor(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim, embedding_dim, batch_size, output_dim):
        super(RecipePredictor, self).__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim)
        self.batch_size = batch_size
        self.predictor = nn.Linear(self.hidden_dim, self.output_dim)
        self.non_linearity = nn.ReLU()
        self.init_hidden() # TODO:  This should happen at the beginning of each epoch
        
    def init_hidden(self):
        self.h_n = torch.randn(1, self.batch_size, self.hidden_dim)
        self.c_n = torch.randn(1, self.batch_size, self.hidden_dim)
        
    
    def forward(self, input_sequence, max_len):
#         print(input_sequence)
#         print(max_len)
        embedded = self.embedding(input_sequence)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, max_len, batch_first=True)
        outputs, (self.c_n, self.h_n) = self.lstm(packed, (self.c_n, self.h_n))
        # Unpack padding
        """
            Honestly, I do not know if at this point, I need the output. I would rather prefer to work with the
            self.h_n cell and so will not `pad_padded_sequence`
        """
        #outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        # on which output should the prediction be done?
        # self.h_n =  num_layers, batch_size, hidden_dim
        batch_size = self.h_n.shape[1]
        output_predicted = self.predictor(self.h_n.reshape(batch_size, -1))
        return output_predicted
        

In [55]:
vocab_size = ingredient_vocabulary.max_idx + 1

In [56]:
num_recipes = recipe_vocabulary.max_idx+1
model = RecipePredictor(vocab_size=vocab_size, hidden_dim=512, embedding_dim=300, batch_size=batch_size,
                       output_dim=num_recipes)

In [57]:
model(X, X_len)

tensor([[ 0.0401,  0.2132,  0.0743, -0.1556,  0.0510,  0.1881, -0.0421,  0.1119,
          0.0713,  0.1193,  0.0347, -0.0798, -0.1863, -0.0639, -0.1612, -0.0156],
        [ 0.0567,  0.0471,  0.0292,  0.0956, -0.0449, -0.0879, -0.0174, -0.1688,
          0.1125, -0.1681,  0.0670, -0.1165,  0.0689,  0.0322, -0.0800,  0.0156],
        [-0.0313,  0.0320, -0.0326,  0.1923,  0.1740, -0.0895, -0.0408,  0.0142,
          0.0165, -0.0118,  0.0636, -0.0233, -0.2884,  0.2130,  0.0235,  0.1105],
        [ 0.1292, -0.0452,  0.1735,  0.2075, -0.1769,  0.0785,  0.1529, -0.1209,
          0.1062,  0.1867,  0.0799, -0.0008,  0.0811, -0.1242,  0.0404, -0.1528],
        [ 0.0271, -0.1232,  0.3260,  0.1704,  0.0706,  0.0453,  0.0249,  0.1153,
          0.2356, -0.0587,  0.2022, -0.1504, -0.0504, -0.0279, -0.0954, -0.0797]],
       grad_fn=<AddmmBackward>)

# Put loss function, gradient and try to train the lstm

In [58]:
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [59]:
num_epoch = 2

In [60]:
# Moving things to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [61]:
model.to(device)

RecipePredictor(
  (embedding): Embedding(1153, 300)
  (lstm): LSTM(300, 512)
  (predictor): Linear(in_features=512, out_features=16, bias=True)
  (non_linearity): ReLU()
)

In [62]:
def put_elements_to_device(a,b,c,device):
    return a.to(device), b.to(device), c.to(device)

In [1]:
# !pip install -q tb-nightly

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [74]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [75]:
print(len(train_data_loader) * batch_size)
print(len(df_train))
# Not same since we are dropping some terms which do not match up
N_train = len(train_data_loader) * batch_size
N_test = len(test_data_loader) * batch_size

79945
79948


In [76]:
for epoch in range(num_epoch):
    running_loss = 0
    model.init_hidden()
    for idx,(X,X_len,y) in enumerate(train_data_loader):
        model.zero_grad()
        X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
        prediction = model(X, X_len)
        loss = criterion(prediction, y)
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if idx % 100 == 0:
            print("epoch {} loss {}".format(epoch, running_loss))
    writer.add_scalar('Loss/train', running_loss/N_train, epoch)
    # validation set is still left to create
    # Now to test the validation set
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for idx,(X,X_len,y) in enumerate(test_data_loader):
            X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
            outputs = model(X,X_len)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += X.size(0)
            correct += (predicted == y).sum().item()
    writer.add_scalar('Loss/train', test_loss/N_test, epoch)
    print('Accuracy of the network on the test samples: %d %%' % (
        100 * correct / total))
        
            

epoch 0 loss 2.0085887908935547
epoch 0 loss 102.51984167844057
epoch 0 loss 221.88543963432312
epoch 0 loss 312.94191985577345
epoch 0 loss 409.68190402537584


KeyboardInterrupt: 

In [67]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [78]:
torch.__version__

'1.3.0'

In [80]:
vindhya_data_df = pd.read_csv('./recipe_title_foodCategory.csv')

In [83]:
list(set(vindhya_data_df.Food_Category))

['Cake',
 'Salad',
 'Noodles',
 'Hummus',
 'Burger',
 'Steak',
 'Pie',
 'Chicken',
 'Custard',
 'Chips',
 'Casserole',
 'Pasta',
 'Pizza',
 'Omelette']

In [94]:
a = [1,2,3,4,5]
b = ['a','b','c','d','e']
c = ['dil','me','dard','e','disco']
sallu_bhoi = zip(a,b,c)

In [95]:
a,b,c = zip(*sallu_bhoi)

In [100]:
np.asarray(a)

array([1, 2, 3, 4, 5])